# Imports and definitions

In [1]:
import itertools
import pathlib
import sys
import time

import pandas
import plotly
import plotly.express
import plotly.graph_objects
import plotly.subplots
import plotly.express.colors

In [2]:
image_classifier_cifar10_resnet18_layer_order = [
    
    "adapter.backbone.conv1",
    "adapter.backbone.bn1",
    "adapter.backbone.relu",
    "adapter.backbone.maxpool",
    
    "adapter.backbone.layer1.0.conv1",
    "adapter.backbone.layer1.0.bn1",
    "adapter.backbone.layer1.0.relu",
    "adapter.backbone.layer1.0.conv2",
    "adapter.backbone.layer1.0.bn2",
    
    "adapter.backbone.layer1.1.conv1",
    "adapter.backbone.layer1.1.bn1",
    "adapter.backbone.layer1.1.relu",
    "adapter.backbone.layer1.1.conv2",
    "adapter.backbone.layer1.1.bn2",
    
    "adapter.backbone.layer2.0.conv1",
    "adapter.backbone.layer2.0.bn1",
    "adapter.backbone.layer2.0.relu",
    "adapter.backbone.layer2.0.conv2",
    "adapter.backbone.layer2.0.bn2",
    "adapter.backbone.layer2.0.downsample.0",
    "adapter.backbone.layer2.0.downsample.1",
    
    "adapter.backbone.layer2.1.conv1",
    "adapter.backbone.layer2.1.bn1",
    "adapter.backbone.layer2.1.relu",
    "adapter.backbone.layer2.1.conv2",
    "adapter.backbone.layer2.1.bn2",
    
    "adapter.backbone.layer3.0.conv1",
    "adapter.backbone.layer3.0.bn1",
    "adapter.backbone.layer3.0.relu",
    "adapter.backbone.layer3.0.conv2",
    "adapter.backbone.layer3.0.bn2",
    "adapter.backbone.layer3.0.downsample.0",
    "adapter.backbone.layer3.0.downsample.1",
    
    "adapter.backbone.layer3.1.conv1",
    "adapter.backbone.layer3.1.bn1",
    "adapter.backbone.layer3.1.relu",
    "adapter.backbone.layer3.1.conv2",
    "adapter.backbone.layer3.1.bn2",
    
    "adapter.backbone.layer4.0.conv1",
    "adapter.backbone.layer4.0.bn1",
    "adapter.backbone.layer4.0.relu",
    "adapter.backbone.layer4.0.conv2",
    "adapter.backbone.layer4.0.bn2",
    "adapter.backbone.layer4.0.downsample.0",
    "adapter.backbone.layer4.0.downsample.1",
    
    "adapter.backbone.layer4.1.conv1",
    "adapter.backbone.layer4.1.bn1",
    "adapter.backbone.layer4.1.relu",
    "adapter.backbone.layer4.1.conv2",
    "adapter.backbone.layer4.1.bn2",
    
    "adapter.backbone.avgpool",
    
    "adapter.head",
]

image_classifier_cifar10_vgg11_layer_order = [f"adapter.backbone.{x}" for x in range(0, 21)] + ["adapter.head"]

# to be used for sorting dataframe layer names
def sorting(reference):
    def key(series):
        unsorted = list(series)
        return pandas.Series([reference.index(el) for el in unsorted])
    return key

experiment_name_conversion_dict = {
    # for random_neuron_bit_weighting in fault aware training
    "False_random": "RBIN",
    "True_random": "RBRN",
    "False_linear": "LBIN",
    "True_linear": "LBRN",
    "False_exponential": "EBIN",
    "True_exponential": "EBRN",
    "False_gradient": "GBIN",
    "True_gradient": "GBRN",
    # for random_neuron_bit_weighting_injection_type in fault aware training
    "False_random_None": "RBIN",
    "True_random_None": "RBRN",
    "False_linear_None": "LBIN",
    "True_linear_None": "LBRN",
    "False_exponential_None": "EBIN",
    "True_exponential_None": "EBRN",
    "False_gradient_None": "GBIN",
    "True_gradient_None": "GBRN",
    "False_random_activation": "RBINo",
    "True_random_activation": "RBRNo",
    "False_linear_activation": "LBINo",
    "True_linear_activation": "LBRNo",
    "False_exponential_activation": "EBINo",
    "True_exponential_activation": "EBRNo",
    "False_gradient_activation": "GBINo",
    "True_gradient_activation": "GBRNo",
    "False_random_weight": "RBINw",
    "True_random_weight": "RBRNw",
    "False_linear_weight": "LBINw",
    "True_linear_weight": "LBRNw",
    "False_exponential_weight": "EBINw",
    "True_exponential_weight": "EBRNw",
    "False_gradient_weight": "GBINw",
    "True_gradient_weight": "GBRNw",
}

configuration_name_to_nice_name_mapping = {
    "vgg11_cifar10": "VGG11<br>CIFAR10",
    "vgg11_gtsrb": "VGG11<br>GTSRB",
    "resnet18_cifar10": "ResNet18<br>CIFAR10",
    "resnet18_gtsrb": "ResNet18<br>GTSRB",
}

experiment_sorting_list = [
    "GBINo", "GBRNo",
    "EBINo", "EBRNo",
    "LBINo", "LBRNo",
    "RBINo", "RBRNo",
    "GBINw", "GBRNw",
    "EBINw", "EBRNw",
    "LBINw", "LBRNw",
    "RBINw", "RBRNw",
]

In [3]:
# max_float = sys.float_info.max
max_float = 2.0e34

metric_names = ["accuracy", "cross_entropy"]

sdc_thresholds = {
    metric_names[0]: list(map(lambda x: x / 100, range(-5, 105, 5))),
    metric_names[1]: list(map(lambda x: 10 ** x, range(-1, 11))),
}
metric_classification_functions = {
    metric_names[0]: {
        "true_positive": lambda baseline, series, threshold: (baseline - series) / baseline >= threshold,
        "false_positive": lambda baseline, series, threshold: (baseline - series) / baseline < threshold,
    },
    metric_names[1]: {
        "true_positive": lambda baseline, series, threshold: (series / baseline) >= threshold,
        "false_positive": lambda baseline, series, threshold: (series / baseline) < threshold,
    },
}

metric_names = metric_names[:1]
sdc_thresholds = {
    metric_names[0]: [0.05, 0.25],
}

# Data Import

## Path Definitions

In [4]:
# base path for IJCNN2023
base_ijcnn2023_result_path = pathlib.Path().cwd() / "../scripts/results/sparse_results_ijcnn2023/"

# # we assume we are in the enpheeph/notebooks folder
# result_paths = {
#     # IJCNN2023, seed 1600, 24-hour time limit (reached before 10e6 injections) is common
#     # comparison between the different algorithms, done on resnet18 gtsrb, weight injection
#     # exponential bit importance neuron
#     "EBIN_resnet18_gtsrb_weight": [
#         base_ijcnn2023_result_path / "importance_sampling/resnet18_gtsrb/epoch-29-step19980_0_pruning/results_injection_type_weight_devices_1_starting_seed_1600_random_threshold_0.0__2023_01_11__00_13_06_+0100/",
#     ],
#     # random bit random neuron
#     "RBRN_resnet18_gtsrb_weight": [
#         base_ijcnn2023_result_path / "random_sampling/resnet18_gtsrb/epoch-29-step19980_0_pruning/results_injection_type_weight_devices_3_starting_seed_1600_random_threshold_1.0__2023_01_11__00_13_46_+0100/",
#     ],
#     # random bit importance neuron
#     "RBIN_resnet18_gtsrb_weight": [
#         base_ijcnn2023_result_path / "random-bit-importance-neuron_sampling/resnet18_gtsrb/epoch-29-step19980_0_pruning/results_injection_type_weight_devices_2_starting_seed_1600_random_threshold_0.0__2023_01_15__16_52_25_+0100/",
#     ],
#     # exponential bit random neuron
#     "EBRN_resnet18_gtsrb_weight": [
#         base_ijcnn2023_result_path / "importance-bit-random-neuron_sampling/resnet18_gtsrb/epoch-29-step19980_0_pruning/results_injection_type_weight_devices_2_starting_seed_1600_random_threshold_1.0__2023_01_12__20_21_45_+0100/",
#     ], 
#     # gradient bit importance neuron
#     "GBIN_resnet18_gtsrb_weight": [
#         base_ijcnn2023_result_path / "gradient-bit-importance-neuron_sampling/resnet18_gtsrb/epoch-29-step19980_0_pruning/results_injection_type_weight_devices_2_starting_seed_1600_random_threshold_0.0_bit_weighting_gradient_sparse_target_None__2023_01_16__01_44_54_+0100/",
#     ],
#     # gradient bit random neuron
#     "GBRN_resnet18_gtsrb_weight": [
#         base_ijcnn2023_result_path / "gradient-bit-random-neuron_sampling/resnet18_gtsrb/epoch-29-step19980_0_pruning/results_injection_type_weight_devices_1_starting_seed_1600_random_threshold_1.0_bit_weighting_gradient_sparse_target_None__2023_01_16__01_43_27_+0100/",
#     ],
# }
# we assume we are in the enpheeph/notebooks folder
# result_paths = {
#     # IJCNN2023, seed 1600, 24-hour time limit (reached before 10e6 injections) is common
#     # comparison between the different algorithms, done on resnet18 gtsrb, activation injection
#     # exponential bit importance neuron
#     "EBIN_resnet18_gtsrb_activation": [
#         base_ijcnn2023_result_path / "importance_sampling/resnet18_gtsrb/epoch-29-step19980_0_pruning/results_injection_type_activation_devices_2_starting_seed_1600_random_threshold_0.0__2023_01_04__01_01_25_+0100/",
#     ],
#     # random bit random neuron
#     "RBRN_resnet18_gtsrb_activation": [
#         base_ijcnn2023_result_path / "random_sampling/resnet18_gtsrb/epoch-29-step19980_0_pruning/results_injection_type_activation_devices_3_starting_seed_1600_random_threshold_1.0__2023_01_05__13_33_35_+0100/",
#     ],
#     # random bit importance neuron
#     "RBIN_resnet18_gtsrb_activation": [
#         base_ijcnn2023_result_path / "random-bit-importance-neuron_sampling/resnet18_gtsrb/epoch-29-step19980_0_pruning/results_injection_type_activation_devices_2_starting_seed_1600_random_threshold_0.0_bit_weighting_random_sparse_target_None__2023_01_18__12_16_36_+0100/",
#     ],
#     # exponential bit random neuron
#     "EBRN_resnet18_gtsrb_activation": [
#         base_ijcnn2023_result_path / "exponential-bit-random-neuron_sampling/resnet18_gtsrb/epoch-29-step19980_0_pruning/results_injection_type_activation_devices_1_starting_seed_1600_random_threshold_1.0_bit_weighting_exponential_sparse_target_None__2023_01_18__12_20_00_+0100/",
#     ],
#     # gradient bit importance neuron
#     "GBIN_resnet18_gtsrb_activation": [
#         base_ijcnn2023_result_path / "gradient-bit-importance-neuron_sampling/resnet18_gtsrb/epoch-29-step19980_0_pruning/results_injection_type_activation_devices_1,2_starting_seed_1600_random_threshold_0.0_bit_weighting_gradient_approximate_activation_gradient_value_True_sparse_target_None__2023_01_22__17_41_47_+0100/",
#     ],
#     # gradient bit random neuron
#     "GBRN_resnet18_gtsrb_activation": [
#         base_ijcnn2023_result_path / "gradient-bit-random-neuron_sampling/resnet18_gtsrb/epoch-29-step19980_0_pruning/results_injection_type_activation_devices_1,2_starting_seed_1600_random_threshold_1.0_bit_weighting_gradient_approximate_activation_gradient_value_True_sparse_target_None__2023_01_22__17_38_49_+0100/",
#     ],
# }
# all the results we have
result_paths = {
    # IJCNN2023, seed 1600, 24-hour time limit (reached before 10e6 injections) is common
    # comparison across all experiments
    
    # resnet18 gtsrb activation
    # exponential bit importance neuron
    "EBINo_resnet18_gtsrb_activation": [
        base_ijcnn2023_result_path / "importance_sampling/resnet18_gtsrb/epoch-29-step19980_0_pruning/results_injection_type_activation_devices_2_starting_seed_1600_random_threshold_0.0__2023_01_04__01_01_25_+0100/",
    ],
    # random bit random neuron
    "RBRNo_resnet18_gtsrb_activation": [
        base_ijcnn2023_result_path / "random_sampling/resnet18_gtsrb/epoch-29-step19980_0_pruning/results_injection_type_activation_devices_3_starting_seed_1600_random_threshold_1.0__2023_01_05__13_33_35_+0100/",
    ],
    # random bit importance neuron
    "RBINo_resnet18_gtsrb_activation": [
        base_ijcnn2023_result_path / "random-bit-importance-neuron_sampling/resnet18_gtsrb/epoch-29-step19980_0_pruning/results_injection_type_activation_devices_2_starting_seed_1600_random_threshold_0.0_bit_weighting_random_sparse_target_None__2023_01_18__12_16_36_+0100/",
    ],
    # exponential bit random neuron
    "EBRNo_resnet18_gtsrb_activation": [
        base_ijcnn2023_result_path / "exponential-bit-random-neuron_sampling/resnet18_gtsrb/epoch-29-step19980_0_pruning/results_injection_type_activation_devices_1_starting_seed_1600_random_threshold_1.0_bit_weighting_exponential_sparse_target_None__2023_01_18__12_20_00_+0100/",
    ],
    # gradient bit importance neuron
    "GBINo_resnet18_gtsrb_activation": [
        base_ijcnn2023_result_path / "gradient-bit-importance-neuron_sampling/resnet18_gtsrb/epoch-29-step19980_0_pruning/results_injection_type_activation_devices_1,2_starting_seed_1600_random_threshold_0.0_bit_weighting_gradient_approximate_activation_gradient_value_True_sparse_target_None__2023_01_22__17_41_47_+0100/",
    ],
    # gradient bit random neuron
    "GBRNo_resnet18_gtsrb_activation": [
        base_ijcnn2023_result_path / "gradient-bit-random-neuron_sampling/resnet18_gtsrb/epoch-29-step19980_0_pruning/results_injection_type_activation_devices_1,2_starting_seed_1600_random_threshold_1.0_bit_weighting_gradient_approximate_activation_gradient_value_True_sparse_target_None__2023_01_22__17_38_49_+0100/",
    ],
    
    # resnet18 gtsrb weight
    # exponential bit importance neuron
    "EBINw_resnet18_gtsrb_weight": [
        base_ijcnn2023_result_path / "importance_sampling/resnet18_gtsrb/epoch-29-step19980_0_pruning/results_injection_type_weight_devices_1_starting_seed_1600_random_threshold_0.0__2023_01_11__00_13_06_+0100/",
    ],
    # random bit random neuron
    "RBRNw_resnet18_gtsrb_weight": [
        base_ijcnn2023_result_path / "random_sampling/resnet18_gtsrb/epoch-29-step19980_0_pruning/results_injection_type_weight_devices_3_starting_seed_1600_random_threshold_1.0__2023_01_11__00_13_46_+0100/",
    ],
    # random bit importance neuron
    "RBINw_resnet18_gtsrb_weight": [
        base_ijcnn2023_result_path / "random-bit-importance-neuron_sampling/resnet18_gtsrb/epoch-29-step19980_0_pruning/results_injection_type_weight_devices_2_starting_seed_1600_random_threshold_0.0__2023_01_15__16_52_25_+0100/",
    ],
    # exponential bit random neuron
    "EBRNw_resnet18_gtsrb_weight": [
        base_ijcnn2023_result_path / "importance-bit-random-neuron_sampling/resnet18_gtsrb/epoch-29-step19980_0_pruning/results_injection_type_weight_devices_2_starting_seed_1600_random_threshold_1.0__2023_01_12__20_21_45_+0100/",
    ], 
    # gradient bit importance neuron
    "GBINw_resnet18_gtsrb_weight": [
        base_ijcnn2023_result_path / "gradient-bit-importance-neuron_sampling/resnet18_gtsrb/epoch-29-step19980_0_pruning/results_injection_type_weight_devices_2_starting_seed_1600_random_threshold_0.0_bit_weighting_gradient_sparse_target_None__2023_01_16__01_44_54_+0100/",
    ],
    # gradient bit random neuron
    "GBRNw_resnet18_gtsrb_weight": [
        base_ijcnn2023_result_path / "gradient-bit-random-neuron_sampling/resnet18_gtsrb/epoch-29-step19980_0_pruning/results_injection_type_weight_devices_1_starting_seed_1600_random_threshold_1.0_bit_weighting_gradient_sparse_target_None__2023_01_16__01_43_27_+0100/",
    ],
    
    # resnet18 cifar10 activation
    # random bit random neuron
    "RBRNo_resnet18_cifar10_activation": [
        base_ijcnn2023_result_path / "random_sampling/resnet18_cifar10/epoch-30-step38750_0_pruning/results_injection_type_activation_devices_2_starting_seed_1600_random_threshold_1.0__2023_01_06__15_49_29_+0100/",
    ],
    # exponential bit importance neuron
    "EBINo_resnet18_cifar10_activation": [
        base_ijcnn2023_result_path / "importance_sampling/resnet18_cifar10/epoch-30-step38750_0_pruning/results_injection_type_activation_devices_1_starting_seed_1600_random_threshold_0.0__2023_01_06__15_48_45_+0100/",
    ],
    
    # resnet18 cifar10 weight
    # random bit random neuron
    "RBRNw_resnet18_cifar10_weight": [
        base_ijcnn2023_result_path / "random_sampling/resnet18_cifar10/epoch-30-step38750_0_pruning/results_injection_type_weight_devices_3_starting_seed_1600_random_threshold_1.0__2023_01_12__00_31_04_+0100/",
    ],
    # random bit importance neuron
    "RBINw_resnet18_cifar10_weight": [
        base_ijcnn2023_result_path / "random-bit-importance-neuron_sampling/resnet18_cifar10/epoch-30-step38750_0_pruning/results_injection_type_weight_devices_1_starting_seed_1600_random_threshold_0.0__2023_01_15__16_51_26_+0100/",
    ],
    # exponential bit importance neuron
    "EBINw_resnet18_cifar10_weight": [
        base_ijcnn2023_result_path / "importance_sampling/resnet18_cifar10/epoch-30-step38750_0_pruning/results_injection_type_weight_devices_1_starting_seed_1600_random_threshold_0.0__2023_01_12__00_30_35_+0100/",
    ],
    # exponential bit random neuron
    "EBRNw_resnet18_cifar10_weight": [
        base_ijcnn2023_result_path / "importance-bit-random-neuron_sampling/resnet18_cifar10/epoch-30-step38750_0_pruning/results_injection_type_weight_devices_1_starting_seed_1600_random_threshold_1.0_bit_weighting_exponential_sparse_target_None__2023_01_16__18_00_01_+0100/",
    ],
    # gradient bit random neuron
    "GBRNw_resnet18_cifar10_weight": [
        base_ijcnn2023_result_path / "gradient-bit-random-neuron_sampling/resnet18_cifar10/epoch-30-step38750_0_pruning/results_injection_type_weight_devices_1_starting_seed_1600_random_threshold_1.0_bit_weighting_gradient_sparse_target_None__2023_01_17__02_14_28_+0100/",
    ],
    # gradient bit importance neuron
    "GBINw_resnet18_cifar10_weight": [
        base_ijcnn2023_result_path / "gradient-bit-importance-neuron_sampling/resnet18_cifar10/epoch-30-step38750_0_pruning/results_injection_type_weight_devices_1_starting_seed_1600_random_threshold_0.0_bit_weighting_gradient_sparse_target_None__2023_01_16__01_44_39_+0100/",
    ],
    
    # vgg11 gtsrb activation
    # random bit random neuron
    "RBRNo_vgg11_gtsrb_activation": [
        base_ijcnn2023_result_path / "random_sampling/vgg11_gtsrb/epoch-29-step19980/results_injection_type_activation_devices_2_starting_seed_1600_random_threshold_1.0__2023_01_07__17_46_03_+0100/",
    ],
    # exponential bit importance neuron
    "EBINo_vgg11_gtsrb_activation": [
        base_ijcnn2023_result_path / "importance_sampling/vgg11_gtsrb/epoch-29-step19980/results_injection_type_activation_devices_3_starting_seed_1600_random_threshold_0.0__2023_01_07__17_45_43_+0100/",
    ],
    
    # vgg11 gtsrb weight
    # random bit random neuron
    "RBRNw_vgg11_gtsrb_weight": [
        base_ijcnn2023_result_path / "random_sampling/vgg11_gtsrb/epoch-29-step19980/results_injection_type_weight_devices_1_starting_seed_1600_random_threshold_1.0__2023_01_13__01_07_14_+0100/",
    ],
    # random bit importance neuron
    "RBINw_vgg11_gtsrb_weight": [
        base_ijcnn2023_result_path / "random-bit-importance-neuron_sampling/vgg11_gtsrb/epoch-29-step19980/results_injection_type_weight_devices_2_starting_seed_1600_random_threshold_0.0_bit_weighting_random_sparse_target_None__2023_01_16__18_00_46_+0100/",
    ],
    # exponential bit importance neuron
    "EBINw_vgg11_gtsrb_weight": [
        base_ijcnn2023_result_path / "importance_sampling/vgg11_gtsrb/epoch-29-step19980/results_injection_type_weight_devices_3_starting_seed_1600_random_threshold_0.0__2023_01_13__01_07_24_+0100/",
    ],
    # exponential bit random neuron
    "EBRNw_vgg11_gtsrb_weight": [
        base_ijcnn2023_result_path / "importance-bit-random-neuron_sampling/vgg11_gtsrb/epoch-29-step19980/results_injection_type_weight_devices_3_starting_seed_1600_random_threshold_1.0_bit_weighting_exponential_sparse_target_None__2023_01_16__22_57_28_+0100/",
    ],
    # gradient bit random neuron
    "GBRNw_vgg11_gtsrb_weight": [
        base_ijcnn2023_result_path / "gradient-bit-random-neuron_sampling/vgg11_gtsrb/epoch-29-step19980/results_injection_type_weight_devices_1_starting_seed_1600_random_threshold_1.0_bit_weighting_gradient_sparse_target_None__2023_01_18__11_26_33_+0100/",
    ],
    # gradient bit importance neuron
    "GBINw_vgg11_gtsrb_weight": [
        base_ijcnn2023_result_path / "gradient-bit-importance-neuron_sampling/vgg11_gtsrb/epoch-29-step19980/results_injection_type_weight_devices_2_starting_seed_1600_random_threshold_0.0_bit_weighting_gradient_sparse_target_None__2023_01_17__02_15_45_+0100/",
    ],
    
    # vgg11 cifar10 activation
    # random bit random neuron
    "RBRNo_vgg11_cifar10_activation": [
        base_ijcnn2023_result_path / "random_sampling/vgg11_cifar10/epoch-4-step6250/results_injection_type_activation_devices_1_starting_seed_1600_random_threshold_1.0__2023_01_09__19_47_52_+0100/",
    ],
    # exponential bit importance neuron
    "EBINo_vgg11_cifar10_activation": [
        base_ijcnn2023_result_path / "importance_sampling/vgg11_cifar10/epoch-4-step6250/results_injection_type_activation_devices_3_starting_seed_1600_random_threshold_0.0__2023_01_08__19_32_51_+0100/",
    ],
    
    # vgg11 cifar10 weight
    # random bit random neuron
    "RBRNw_vgg11_cifar10_weight": [
        base_ijcnn2023_result_path / "random_sampling/vgg11_cifar10/epoch-4-step6250/results_injection_type_weight_devices_2_starting_seed_1600_random_threshold_1.0__2023_01_14__02_52_18_+0100/",
    ],
    # random bit importance neuron
    "RBINw_vgg11_cifar10_weight": [
        base_ijcnn2023_result_path / "random-bit-importance-neuron_sampling/vgg11_cifar10/epoch-4-step6250/results_injection_type_weight_devices_3_starting_seed_1600_random_threshold_0.0_bit_weighting_random_sparse_target_None__2023_01_16__22_58_16_+0100/",
    ],
    # exponential bit importance neuron
    "EBINw_vgg11_cifar10_weight": [
        base_ijcnn2023_result_path / "importance_sampling/vgg11_cifar10/epoch-4-step6250/results_injection_type_weight_devices_1_starting_seed_1600_random_threshold_0.0__2023_01_14__02_51_57_+0100/",
    ],
    # exponential bit random neuron
    "EBRNw_vgg11_cifar10_weight": [
        base_ijcnn2023_result_path / "importance-bit-random-neuron_sampling/vgg11_cifar10/epoch-4-step6250/results_injection_type_weight_devices_1_starting_seed_1600_random_threshold_1.0_bit_weighting_exponential_sparse_target_None__2023_01_18__11_23_38_+0100/",
    ],
    # gradient bit random neuron
    "GBRNw_vgg11_cifar10_weight": [
        base_ijcnn2023_result_path / "gradient-bit-random-neuron_sampling/vgg11_cifar10/epoch-4-step6250/results_injection_type_weight_devices_2_starting_seed_1600_random_threshold_1.0_bit_weighting_gradient_sparse_target_None__2023_01_18__11_26_44_+0100/",
    ],
    # gradient bit importance neuron
    "GBINw_vgg11_cifar10_weight": [
        base_ijcnn2023_result_path / "gradient-bit-importance-neuron_sampling/vgg11_cifar10/epoch-4-step6250/results_injection_type_weight_devices_1_starting_seed_1600_random_threshold_0.0_bit_weighting_gradient_sparse_target_None__2023_01_17__02_14_42_+0100/",
    ],
    
}
# we want to compare all the resnet18 gtsrb weight with the different values of random-importance
# result_path = result_paths["importance_vgg11_cifar10_activation"]

## CSV Parsing

In [5]:
csv_list = {}
for name, paths in result_paths.items():
    original_csvs = []
    csv = pandas.DataFrame()
    for path in paths:
        for csv_file in path.glob("**/*.csv"):
            new_csv = pandas.read_csv(
                csv_file,
                sep=",",
                header=0,
                index_col=0,
                # dtype=str,
                engine="python",
                # nrows=3400,
                on_bad_lines="warn",
            )
            csv = pandas.concat(
                [
                    csv,
                    new_csv.reset_index()
                ],
                axis=0,
            )
            original_csvs.append(new_csv.copy())
    csv_list[name] = {"csv": csv.reset_index(drop=True), "original_csvs": original_csvs}

## Data Processing

In [6]:
# csv.sort_values("test_cross_entropy_injected", ascending=False)
for name, csv_dict in csv_list.copy().items():
    print(name, csv_dict["csv"].columns)
    csv = csv_dict["csv"]
    
    # substitute inf and nan with max representable number
    for metric_name in metric_names:
        csv[f"test_{metric_name}_injected"] = csv[f"test_{metric_name}_injected"].replace(float("inf"), max_float)
        csv[f"test_{metric_name}_injected"] = csv[f"test_{metric_name}_injected"].replace(float("-inf"), -max_float)
    csv["test_accuracy_injected"] = csv["test_accuracy_injected"].replace(float("NaN"), 0.0)
    csv["test_cross_entropy_injected"] = csv["test_cross_entropy_injected"].replace(float("NaN"), max_float)
    
    csv_list[name]["csv"] = csv
csv.head()

EBINo_resnet18_gtsrb_activation Index(['i', 'random_seed', 'random_threshold', 'timestamp', 'target_csv',
       'injection_type', 'extra_injection_info', 'layer_name', 'index',
       'bit_index', 'random', 'test_accuracy_baseline',
       'test_cross_entropy_baseline', 'test_accuracy_injected',
       'test_cross_entropy_injected'],
      dtype='object')
RBRNo_resnet18_gtsrb_activation Index(['i', 'random_seed', 'random_threshold', 'timestamp', 'target_csv',
       'injection_type', 'extra_injection_info', 'layer_name', 'index',
       'bit_index', 'random', 'test_accuracy_baseline',
       'test_cross_entropy_baseline', 'test_accuracy_injected',
       'test_cross_entropy_injected'],
      dtype='object')
RBINo_resnet18_gtsrb_activation Index(['i', 'random_seed', 'random_threshold', 'timestamp', 'target_csv',
       'injection_type', 'extra_injection_info', 'layer_name', 'index',
       'bit_index', 'bit_weighting', 'random_neuron', 'test_accuracy_baseline',
       'test_cross_entro

,i,random_seed,random_threshold,timestamp,target_csv,injection_type,extra_injection_info,layer_name,index,bit_index,bit_weighting,random_neuron,test_accuracy_baseline,test_cross_entropy_baseline,test_accuracy_injected,test_cross_entropy_injected
0,0,1600,0.0,2023-01-17T01:15:23.674789+0000,/home/acolucci/Documents/PhD/Projects/FaultInj...,weight,bit_weighting+gradient,adapter.backbone.0,9-1-0-0,29,gradient,False,0.818,0.596383,0.8154,6.047016e-01
1,1,1600,0.0,2023-01-17T01:15:30.990739+0000,/home/acolucci/Documents/PhD/Projects/FaultInj...,weight,bit_weighting+gradient,adapter.head,9-429,30,gradient,False,NaN,NaN,0.5830,2.000000e+34
2,2,1600,0.0,2023-01-17T01:15:46.801701+0000,/home/acolucci/Documents/PhD/Projects/FaultInj...,weight,bit_weighting+gradient,adapter.backbone.8,154-159-2-0,29,gradient,False,NaN,NaN,0.8180,5.963845e-01
3,3,1600,0.0,2023-01-17T01:16:22.973831+0000,/home/acolucci/Documents/PhD/Projects/FaultInj...,weight,bit_weighting+gradient,adapter.backbone.13,454-153-2-0,28,gradient,False,NaN,NaN,0.8180,5.963827e-01
4,4,1600,0.0,2023-01-17T01:16:41.962498+0000,/home/acolucci/Documents/PhD/Projects/FaultInj...,weight,bit_weighting+gradient,adapter.backbone.11,295-114-1-1,30,gradient,False,NaN,NaN,0.8179,5.969548e-01


In [7]:
# we want to compute the precision and the recall of the injection
# we need to add columns for labelling each injection as true positive or false positive, based on the
# SDC/DUE threshold

# we assume all the faults can be categorized as positive, either true if accuracy drop is bigger than threshold
# or false if below

for name, csv_dict in csv_list.copy().items():
    csv = csv_dict["csv"]
    
    for metric_name in metric_names:
        baseline = csv[f"test_{metric_name}_baseline"].dropna().tolist()[0]
        for i, sdc_threshold in enumerate(sdc_thresholds[metric_name]):
            true_positive = metric_classification_functions[metric_name]["true_positive"](baseline, csv[f"test_{metric_name}_injected"], sdc_threshold)
            false_positive = metric_classification_functions[metric_name]["false_positive"](baseline, csv[f"test_{metric_name}_injected"], sdc_threshold)
            precision = pandas.Series([true_positive[:i+1].sum() / (true_positive[:i+1].sum() + false_positive[:i+1].sum()) for i in range(len(true_positive))]) 
            df = pandas.DataFrame({
                f"SDC_{metric_name}_{sdc_threshold}_precision": precision.reindex(),
                f"SDC_{metric_name}_{sdc_threshold}_true_positive": true_positive.reindex(),
                f"SDC_{metric_name}_{sdc_threshold}_false_positive": false_positive.reindex(),
            })
            csv = pandas.concat([csv, df], axis=1)
            # to avoid fragmentation
            csv = csv.copy()
    csv_list[name]["csv"] = csv

In [8]:
for name, csv_dict in csv_list.copy().items():
    csv = csv_dict["csv"]
    
    # substitute inf and nan with max representable number
    csv = csv.replace(float("NaN"), max_float)
    csv = csv.replace(float("inf"), max_float)
    csv = csv.replace(float("-inf"), -max_float)
    
    csv_list[name]["csv"] = csv

In [9]:
for name, csv_dict in csv_list.copy().items():
    csv = csv_dict["csv"]
    for metric_name in metric_names:
        for i, sdc_threshold in enumerate(sdc_thresholds[metric_name]): 
            print(f"{name}, SDC_{metric_name}_{sdc_threshold}_precision: {csv[f'SDC_{metric_name}_{sdc_threshold}_precision'].tolist()[-1]}")

EBINo_resnet18_gtsrb_activation, SDC_accuracy_0.05_precision: 0.4055354993983153
EBINo_resnet18_gtsrb_activation, SDC_accuracy_0.25_precision: 0.3536569060034764
RBRNo_resnet18_gtsrb_activation, SDC_accuracy_0.05_precision: 0.10219731603008406
RBRNo_resnet18_gtsrb_activation, SDC_accuracy_0.25_precision: 0.0737354372511429
RBINo_resnet18_gtsrb_activation, SDC_accuracy_0.05_precision: 0.056945481702763256
RBINo_resnet18_gtsrb_activation, SDC_accuracy_0.25_precision: 0.03472740851381628
EBRNo_resnet18_gtsrb_activation, SDC_accuracy_0.05_precision: 0.25841476655808904
EBRNo_resnet18_gtsrb_activation, SDC_accuracy_0.25_precision: 0.18512486427795874
GBINo_resnet18_gtsrb_activation, SDC_accuracy_0.05_precision: 0.5817864989771953
GBINo_resnet18_gtsrb_activation, SDC_accuracy_0.25_precision: 0.42662322903250244
GBRNo_resnet18_gtsrb_activation, SDC_accuracy_0.05_precision: 0.4897677627040699
GBRNo_resnet18_gtsrb_activation, SDC_accuracy_0.25_precision: 0.34620985667203186
EBINw_resnet18_gtsrb

In [10]:
# conversion_dict = {"importance": "IBIN", "random": "RBRN", "importance-bit-random-neuron": "IBRN", "random-bit-importance-neuron": "RBIN"}
sdc_alltogether_histogram_csv_dict = {"experiment": [], "configuration": [], "configuration_nice": [], "precision": [], "accuracy_delta": []}
for name, csv_dict in csv_list.copy().items():
    csv = csv_dict["csv"]
    for metric_name in metric_names:
#         if i % 3 != 1:
#             continue
#         sdc_csv_dict["experiment"].append(conversion_dict[name.split("_")[0]])
        for i, sdc_threshold in enumerate(sdc_thresholds[metric_name]):
#             sdc_histogram_csv_dict["experiment"].append(name.split("_")[0] + f"_{metric_name}")
            sdc_alltogether_histogram_csv_dict["experiment"].append(name.split("_")[0])
            sdc_alltogether_histogram_csv_dict["configuration"].append("_".join(name.split("_")[1:3]))
            sdc_alltogether_histogram_csv_dict["configuration_nice"].append(configuration_name_to_nice_name_mapping[sdc_alltogether_histogram_csv_dict["configuration"][-1]])
            sdc_alltogether_histogram_csv_dict["precision"].append(csv[f'SDC_{metric_name}_{sdc_threshold}_precision'].tolist()[-1])
            sdc_alltogether_histogram_csv_dict["accuracy_delta"].append(sdc_threshold)
sdc_alltogether_histogram_csv = pandas.DataFrame(sdc_alltogether_histogram_csv_dict)
sdc_alltogether_histogram_csv.head()

,experiment,configuration,configuration_nice,precision,accuracy_delta
0,EBINo,resnet18_gtsrb,ResNet18<br>GTSRB,0.405535,0.05
1,EBINo,resnet18_gtsrb,ResNet18<br>GTSRB,0.353657,0.25
2,RBRNo,resnet18_gtsrb,ResNet18<br>GTSRB,0.102197,0.05
3,RBRNo,resnet18_gtsrb,ResNet18<br>GTSRB,0.073735,0.25
4,RBINo,resnet18_gtsrb,ResNet18<br>GTSRB,0.056945,0.05


In [11]:
sdc_alltogether_histogram_csv = sdc_alltogether_histogram_csv.sort_values(by="experiment", key=lambda x: [experiment_sorting_list.index(y) for y in x])

In [12]:
experiments_to_plot = ["EBINo", "RBRNo", "EBINw", "RBRNw"]
flag = None
for e in experiments_to_plot:
    if flag is None:
        flag = sdc_alltogether_histogram_csv["experiment"] == e
    else:
        flag = flag | (sdc_alltogether_histogram_csv["experiment"] == e)
# sdc_alltogether_histogram_csv = sdc_alltogether_histogram_csv[flag]

# Plotting

In [16]:
# df = plotly.express.data.tips()
# fig = plotly.express.bar(df, x="day", y="total_bill", color="smoker", barmode="group", facet_col="sex",
#              category_orders={"day": ["Thur", "Fri", "Sat", "Sun"],
#                               "smoker": ["Yes", "No"],
#                               "sex": ["Male", "Female"]})
# fig.show()
fig = plotly.express.histogram(
    sdc_alltogether_histogram_csv,
    x="configuration_nice",
    y="precision",
    color="experiment",
    barmode="group",
    histfunc="avg",
    labels=dict(precision="Precision", experiment=""),
    text_auto=".3f",
    # pattern_shape="experiment",
    # color_discrete_sequence=plotly.express.colors.colorbrewer.PuOr[::2],
    color_discrete_sequence=plotly.express.colors.cmocean.haline,
)
fig.update_layout(
    barmode="group",
    xaxis={"title": ''},
    yaxis={"title": "Precision"},
    
    font={'size': 24},
    template="presentation",
    margin=dict(l=70, t=20, b=70, r=0),
    legend=dict(
        x=0.1,  # value must be between 0 to 1.
        y=1,   # value must be between 0 to 1.
        bordercolor="Black",
        borderwidth=1,
    ),
#     annotations=[
#          dict(
#              text="G",
#              x=0.40,
#              ax=0.35,
#              y=0.15,
#              ay=0.4,
#              xref="x domain",
#              axref="x domain",
#              yref="y domain",
#              ayref="y domain",
# #              bgcolor="rgba(255, 255, 255, 0)",
# #              bordercolor="rgba(255, 255, 255, 0)",
#              bgcolor="green",
#              bordercolor='green',
#              borderwidth=3,
#              borderpad=0,
#              font=dict(
#                  color="white",
#                  size=50,
#              ),
#              arrowhead=2,
#              arrowsize=1.2,
#              arrowwidth=3,
#              arrowcolor="Green",
#          ),
#     ],
#     shapes=[
#         dict(
#             type="rect",
#             xref="x domain",
#             yref="y domain",
#             x0=0.31,
#             x1=0.5,
#             y0=0.03,
#             y1=0.15,
#             line=dict(
#                 color="Green",
#                 width=3,
#             ),
#         ),
# #         dict(
# #             type="circle",
# #             xref="x domain",
# #             yref="y domain",
# #             x0=0.4,
# #             x1=0.5,
# #             y0=0.15,
# #             y1=0.25,
# #             fillcolor="green",
# #             line=dict(
# #                 color="Green",
# #                 width=1,
# #             ),
# #         ),
#     ],
#     annotationdefaults=dict(
        
#     ),
)
# fig.write_html(base_ijcnn2023_result_path / "plots/histogram_plot_sdc.html")
fig.show()

In [14]:
# for ____ in range(2):
#     fig.write_image(file=base_ijcnn2023_result_path / "plots/alltogether_histogram.pdf", format='pdf', height=360)
#     fig.write_image(file=base_ijcnn2023_result_path / "plots/alltogether_histogram.svg", format='svg', height=360)
#     time.sleep(2)

In [15]:
plotly.express.colors.cmocean.haline

['rgb(41, 24, 107)',
 'rgb(42, 35, 160)',
 'rgb(15, 71, 153)',
 'rgb(18, 95, 142)',
 'rgb(38, 116, 137)',
 'rgb(53, 136, 136)',
 'rgb(65, 157, 133)',
 'rgb(81, 178, 124)',
 'rgb(111, 198, 107)',
 'rgb(160, 214, 91)',
 'rgb(212, 225, 112)',
 'rgb(253, 238, 153)']